In [0]:
#import
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
#import lightgbm as lgb
from scipy import stats
import matplotlib.pyplot as plt
from numpy import mean
from numpy import std
import math
import scipy
from scipy.stats import stats
from numpy.random import choice
import random

In [0]:
from google.colab import drive 
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
map_object_cols={}

map_object_cols['ProductCD']='object'
map_object_cols['DeviceInfo']='object'
map_object_cols['DeviceType']='object'
map_object_cols['addr1']='object'
map_object_cols['addr2']='object'
map_object_cols['device_name']='object'
map_object_cols['had_id']='object'

map_object_cols['P_emaildomain']='object'
map_object_cols['P_emaildomain_bin']='object'
map_object_cols['P_emaildomain_suffix']='object'
map_object_cols['R_emaildomain']='object'
map_object_cols['R_emaildomain_bin']='object'
map_object_cols['R_emaildomain_suffix']='object'

map_object_cols['_Month']='object'
map_object_cols['_Weekdays']='object'
map_object_cols['_Days']='object'
map_object_cols['_Hours']='object'

for i in range(12,39):
  col_name= 'id_'+str(i)
  map_object_cols[col_name]='object'

for i in range(1,10):
  col_name= 'M'+str(i)
  map_object_cols[col_name]='object'

for i in range(1,7):
  col_name= 'card'+str(i)
  map_object_cols[col_name]='object'

In [0]:
train=pd.read_csv('gdrive/My Drive/Colab Notebooks/Fraud/Data/fraud_data_filteredColumnsWithHigherThank85PercentMissing.csv',compression='gzip',dtype=map_object_cols)

In [0]:
train.shape

(590540, 368)

In [0]:
count_integers=len(train.select_dtypes(include=['integer']).columns)
print('There are ' + str(count_integers) +' columns of type integer')
count_float=len(train.select_dtypes(include=['floating']).columns)
print('There are ' + str(count_float) +' columns of type float')
count_category=len(train.select_dtypes(include=['object']).columns)
print('There are ' + str(count_category) +' columns of type object')
print('Total ' + str(count_category+count_float+count_integers))

There are 1 columns of type integer
There are 321 columns of type float
There are 46 columns of type object
Total 368


In [0]:
train

,isFraud,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D10,D11,D15,M1,M2,M3,M4,...,V313,V314,V315,V316,V317,V318,V319,V320,V321,id_01,id_02,id_05,id_06,id_11,id_12,id_13,id_15,id_16,id_17,id_19,id_20,id_28,id_29,id_31,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo,_Weekdays,_Hours,_Days,P_emaildomain_bin,P_emaildomain_suffix,R_emaildomain_bin,R_emaildomain_suffix,device_name,had_id,_Month
0,0,68.50,W,13926,NaN,150.0,discover,142.0,credit,315.0,87.0,19.0,NaN,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,NaN,13.0,NaN,NaN,13.0,13.0,0.0,T,T,T,M2,...,0.000000,0.000000,0.000000,0.0,117.0,0.0,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,0,2,NaN,NaN,NaN,NaN,NaN,1,12
1,0,29.00,W,2755,404.0,150.0,mastercard,102.0,credit,325.0,87.0,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,M0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,0,2,google,com,NaN,NaN,NaN,1,12
2,0,59.00,W,4663,490.0,150.0,visa,166.0,debit,330.0,87.0,287.0,outlook.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,0.0,315.0,315.0,T,T,T,M0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,0,2,microsoft,com,NaN,NaN,NaN,1,12
3,0,50.00,W,18132,567.0,150.0,mastercard,117.0,debit,476.0,87.0,NaN,yahoo.com,NaN,2.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,25.0,1.0,112.0,112.0,0.0,94.0,0.0,84.0,NaN,111.0,NaN,NaN,NaN,M0,...,0.000000,0.000000,0.000000,50.0,1404.0,790.0,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,0,2,yahoo,com,NaN,NaN,NaN,1,12
4,0,50.00,H,4497,514.0,150.0,mastercard,102.0,credit,420.0,87.0,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,70787.0,NaN,NaN,100.0,NotFound,NaN,New,NotFound,166.0,542.0,144.0,New,NotFound,Samsung,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M,5,0,2,google,com,NaN,NaN,Samsung,1,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,0,49.00,W,6550,NaN,150.0,visa,226.0,debit,272.0,87.0,48.0,NaN,NaN,2.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,3.0,2.0,29.0,29.0,30.0,NaN,NaN,56.0,56.0,56.0,T,T,T,M0,...,47.950001,47.950001,47.950001,0.0,0.0,0.0,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,23,1,NaN,NaN,NaN,NaN,NaN,1,6
590536,0,39.50,W,10444,225.0,150.0,mastercard,224.0,debit,204.0,87.0,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,0.0,0.0,0.0,T,F,F,M0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,23,1,google,com,NaN,NaN,NaN,1,6
590537,0,30.95,W,12037,595.0,150.0,mastercard,224.0,debit,231.0,87.0,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,0.0,0.0,0.0,T,F,F,NaN,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,23,1,google,com,NaN,NaN,NaN,1,6
590538,0,117.00,W,7826,481.0,150.0,mastercard,224.0,debit,387.0,87.0,3.0,aol.com,NaN,1.0,1.0,0.0,0.0,0.0,3.0,0.0,0.0,2.0,0.0,1

In [0]:
train.isnull().sum().sum()

78497545

In [0]:
train3=train.copy()

In [0]:
int_cols=set(train.select_dtypes(include=['integer']).columns)
int_cols

{'isFraud'}

In [0]:
int_cols.remove('isFraud')

In [0]:
float_cols = train.select_dtypes(include=['floating']).columns
len(float_cols)

321

In [0]:
float_to_int_cols=set()
for col in float_cols:
  col_df = train[col].dropna()
  col_should_be_int = col_df.map(float.is_integer).all() 
  if col_should_be_int:
    float_to_int_cols.add(col)

In [0]:
len(float_to_int_cols)

260

In [0]:
 int_cols=int_cols.union(float_to_int_cols)
 len(int_cols)

260

In [0]:
df_integers= train[list(int_cols)]

In [0]:
df_integers.shape

(590540, 260)

In [0]:
df_integers

,V36,V219,V1,V60,V85,V122,V288,V255,V79,V56,V257,C14,D15,V52,V172,V186,C7,D10,V201,V41,V94,V236,D1,V9,C4,C9,V65,V53,V61,V242,V293,V305,V4,V32,V84,V229,V39,V38,V23,V249,...,V111,V170,V3,V62,V27,V99,V180,V123,V223,V237,V40,V121,V17,V66,V110,V302,V199,id_05,V290,V7,V74,V234,V42,V89,V233,V87,V193,V15,V51,V69,V72,V21,V101,V169,V240,V241,V119,V109,V2,V185
0,NaN,NaN,1.0,0.0,0.0,1.0,0.0,NaN,0.0,1.0,NaN,1.0,0.0,NaN,NaN,NaN,0.0,13.0,NaN,NaN,0.0,NaN,14.0,1.0,0.0,1.0,1.0,1.0,1.0,NaN,0.0,1.0,1.0,0.0,0.0,NaN,NaN,NaN,1.0,NaN,...,1.0,NaN,1.0,1.0,0.0,0.0,NaN,1.0,NaN,NaN,NaN,1.0,0.0,1.0,1.0,0.0,NaN,NaN,1.0,1.0,0.0,NaN,NaN,0.0,NaN,1.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,1.0,1.0,1.0,NaN
1,0.0,NaN,NaN,0.0,0.0,1.0,0.0,NaN,0.0,1.0,NaN,1.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,1.0,0.0,NaN,0.0,NaN,0.0,0.0,1.0,0.0,1.0,NaN,0.0,1.0,NaN,0.0,0.0,NaN,0.0,1.0,1.0,NaN,...,1.0,NaN,NaN,1.0,0.0,0.0,NaN,1.0,NaN,NaN,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,1.0,NaN,0.0,NaN,0.0,0.0,NaN,1.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,1.0,1.0,NaN,NaN
2,1.0,NaN,1.0,0.0,0.0,1.0,0.0,NaN,0.0,1.0,NaN,1.0,315.0,0.0,NaN,NaN,0.0,0.0,NaN,1.0,0.0,NaN,0.0,1.0,0.0,1.0,1.0,1.0,1.0,NaN,0.0,1.0,1.0,0.0,0.0,NaN,0.0,1.0,1.0,NaN,...,1.0,NaN,1.0,1.0,0.0,0.0,NaN,1.0,NaN,NaN,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,1.0,1.0,0.0,NaN,0.0,0.0,NaN,1.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,1.0,1.0,1.0,NaN
3,1.0,NaN,NaN,0.0,0.0,1.0,0.0,NaN,0.0,1.0,NaN,1.0,111.0,0.0,NaN,NaN,0.0,84.0,NaN,1.0,0.0,NaN,112.0,NaN,0.0,1.0,1.0,1.0,1.0,NaN,1.0,1.0,NaN,0.0,0.0,NaN,0.0,1.0,1.0,NaN,...,1.0,NaN,NaN,1.0,0.0,10.0,NaN,1.0,NaN,NaN,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,1.0,NaN,0.0,NaN,0.0,0.0,NaN,1.0,NaN,0.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,1.0,1.0,NaN,NaN
4,NaN,0.0,NaN,NaN,NaN,1.0,0.0,1.0,NaN,NaN,1.0,1.0,NaN,NaN,0.0,1.0,0.0,NaN,1.0,NaN,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,1.0,0.0,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,...,1.0,1.0,NaN,NaN,NaN,0.0,0.0,1.0,0.0,0.0,NaN,1.0,NaN,NaN,1.0,1.0,1.0,NaN,1.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,1.0,1.0,1.0,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,NaN,NaN,1.0,0.0,0.0,1.0,1.0,NaN,0.0,1.0,NaN,2.0,56.0,NaN,NaN,NaN,0.0,56.0,NaN,NaN,0.0,NaN,29.0,1.0,0.0,2.0,1.0,1.0,0.0,NaN,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,1.0,NaN,...,1.0,NaN,1.0,0.0,0.0,1.0,NaN,1.0,NaN,NaN,NaN,1.0,0.0,1.0,1.0,0.0,NaN,NaN,1.0,1.0,0.0,NaN,NaN,0.0,NaN,1.0,NaN,0.0,NaN,1.0,0.0,0.0,0.0,NaN,NaN,NaN,1.0,1.0,1.0,NaN
590536,1.0,NaN,1.0,0.0,0.0,1.0,0.0,NaN,0.0,1.0,NaN,1.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,1.0,0.0,NaN,0.0,1.0,0.0,1.0,1.0,1.0,1.0,NaN,0.0,1.0,1.0,0.0,0.0,NaN,0.0,1.0,1.0,NaN,...,1.0,NaN,1.0,1.0,0.0,0.0,NaN,1.0,NaN,NaN,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,1.0,1.0,0.0,NaN,0.0,0.0,NaN,1.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,1.0,1.0,1.0,NaN
590537,1.0,NaN,1.0,0.0,0.0,1.0,0.0,NaN,0.0,1.0,NaN,1.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,1.0,0.0,NaN,0.0,1.0,0.0,1.0,1.0,1.0,1.0,NaN,0.0,1.0,1.0,0.0,0.0,NaN,0.0,1.0,1.0,NaN,...,1.0,NaN,1.0,1.0,0.0,0.0,NaN,1.0,NaN,NaN,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,1.0,1.0,0.0,NaN,0.0,0.0,NaN,1.0,NaN,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,1.0,1.0,1.0,NaN
590538,1.0,NaN,1.0,0.0,0.0,1.0,1.0,NaN,0.0,1.0,NaN,1.0,22.0,1.0,NaN,NaN,0.0,22.0,NaN,1.0,0.0,NaN,22.0,1.0,0.0,2.0,1.0,1.0,2.0,NaN,0.0,1.0,2.0,0.0,0.0,NaN,0.0,1.0,1.0,NaN,...,1.0,NaN,1.0,2.0,0.0,1.0,NaN,1.0,NaN,NaN,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,1.0,1.0,1.0,NaN,0.0,0.0,NaN,1.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,1.0,1.0,1.0,NaN


In [0]:
len(df_integers)

590540

In [0]:
train3=train.copy()

In [0]:
#values=df_integers['V36'].dropna().unique().tolist()
df_integers_dropna=df_integers['V36'].dropna()
values=df_integers_dropna.unique().tolist()
values__probs = df_integers.groupby('V36').size().div(len(df_integers_dropna))
randomnumber = choice(values, p=values__probs)
randomnumber

0.0

In [0]:
for col in int_cols:
  values=df_integers[col].unique().tolist()
  values__probs = df_integers.groupby(col).size().div(len(df_integers))
  randomnumber = choice(values, p=values__probs)
  train4[col] = train4[col].fillna(randomnumber)

In [0]:
# TODO: check if this code is ok
#for col in int_cols:
#  df_integers_dropna=df_integers[col].dropna()
#  values=df_integers_dropna.unique().tolist()
  #print('values size: '+ str(len(values)))
#  values__probs = df_integers.groupby(col).size().div(len(df_integers_dropna))
  #print('values prob size: '+ str(len(values__probs)))
  #randomnumber = choice(values, p=values__probs)
  
#  df_nan=train.loc[(train[col].isnull())]
#  indexes=set(df_nan.index)

  #values_list=random.sample(choice(values, p=values__probs),k=len(indexes))
#  values_to_fill={}
#  n=0;
#  for i in indexes:
#    values_to_fill[i]=choice(values, p=values__probs)
#    n+=1

#  train3[col] = train3[col].fillna(value=values_to_fill)

#train3[col] = train3[col].fillna(randomnumber)

In [0]:
train3.head()

,isFraud,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D10,D11,D15,M1,M2,M3,M4,...,V313,V314,V315,V316,V317,V318,V319,V320,V321,id_01,id_02,id_05,id_06,id_11,id_12,id_13,id_15,id_16,id_17,id_19,id_20,id_28,id_29,id_31,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo,_Weekdays,_Hours,_Days,P_emaildomain_bin,P_emaildomain_suffix,R_emaildomain_bin,R_emaildomain_suffix,device_name,had_id,_Month
0,0,68.5,W,13926,NaN,150.0,discover,142.0,credit,315.0,87.0,19.0,NaN,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,552.0,13.0,290.0,222.0,13.0,13.0,0.0,T,T,T,M2,...,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,-51.0,105328.0,26.0,-89.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,0,2,NaN,NaN,NaN,NaN,NaN,1,12
1,0,29.0,W,2755,404.0,150.0,mastercard,102.0,credit,325.0,87.0,154.0,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,112.0,13.0,0.0,46.0,0.0,320.0,0.0,NaN,NaN,NaN,M0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-56.0,353319.0,26.0,-90.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,0,2,google,com,NaN,NaN,NaN,1,12
2,0,59.0,W,4663,490.0,150.0,visa,166.0,debit,330.0,87.0,287.0,outlook.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,61.0,13.0,0.0,11.0,0.0,315.0,315.0,T,T,T,M0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-96.0,443506.0,-27.0,-89.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,0,2,microsoft,com,NaN,NaN,NaN,1,12
3,0,50.0,W,18132,567.0,150.0,mastercard,117.0,debit,476.0,87.0,287.0,yahoo.com,NaN,2.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,25.0,1.0,112.0,112.0,0.0,94.0,0.0,84.0,118.0,111.0,NaN,NaN,NaN,M0,...,0.0,0.0,0.0,50.0,1404.0,790.0,0.0,0.0,0.0,-8.0,843852.0,-7.0,-83.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,0,2,yahoo,com,NaN,NaN,NaN,1,12
4,0,50.0,H,4497,514.0,150.0,mastercard,102.0,credit,420.0,87.0,13.0,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,128.0,11.0,463.0,214.0,185.0,245.0,268.0,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,70787.0,29.0,-89.0,100.0,NotFound,NaN,New,NotFound,166.0,542.0,144.0,New,NotFound,Samsung,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M,5,0,2,google,com,NaN,NaN,Samsung,1,12


In [0]:
float_cols=set(train.select_dtypes(include=['floating']).columns) - int_cols
len(float_cols)

61

In [0]:
for col in float_cols:
    vals=pd.Series(train[col][train[col].notnull()])
    #print(vals)
    density = scipy.stats.gaussian_kde(vals)
    df_nan=train.loc[(train[col].isnull())]
    indexes=set(df_nan.index)
    sample=density.resample(len(indexes)).T[:,0]
    values_list=random.sample(set(pd.Series(sample)),k=len(indexes))
    values_to_fill={}
    n=0;
    for i in indexes:
        values_to_fill[i]=values_list[n]
        n+=1

    train3[col] = train3[col].fillna(value=values_to_fill)

In [0]:
train3.isnull().sum().sum()

12160821

In [0]:
train3

,isFraud,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D10,D11,D15,M1,M2,M3,M4,...,V313,V314,V315,V316,V317,V318,V319,V320,V321,id_01,id_02,id_05,id_06,id_11,id_12,id_13,id_15,id_16,id_17,id_19,id_20,id_28,id_29,id_31,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo,_Weekdays,_Hours,_Days,P_emaildomain_bin,P_emaildomain_suffix,R_emaildomain_bin,R_emaildomain_suffix,device_name,had_id,_Month
0,0,68.50,W,13926,NaN,150.0,discover,142.0,credit,315.0,87.0,19.0,NaN,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,552.0,13.0,290.0,222.0,13.0,13.0,0.0,T,T,T,M2,...,0.000000,0.000000,0.000000,0.0,117.0,0.0,0.000000,0.000000,0.000000,-51.0,105328.0,26.0,-89.0,95.065027,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,0,2,NaN,NaN,NaN,NaN,NaN,1,12
1,0,29.00,W,2755,404.0,150.0,mastercard,102.0,credit,325.0,87.0,154.0,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,112.0,13.0,0.0,46.0,0.0,320.0,0.0,NaN,NaN,NaN,M0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,-56.0,353319.0,26.0,-90.0,99.962037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,0,2,google,com,NaN,NaN,NaN,1,12
2,0,59.00,W,4663,490.0,150.0,visa,166.0,debit,330.0,87.0,287.0,outlook.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,61.0,13.0,0.0,11.0,0.0,315.0,315.0,T,T,T,M0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,-96.0,443506.0,-27.0,-89.0,100.024600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,0,2,microsoft,com,NaN,NaN,NaN,1,12
3,0,50.00,W,18132,567.0,150.0,mastercard,117.0,debit,476.0,87.0,287.0,yahoo.com,NaN,2.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,25.0,1.0,112.0,112.0,0.0,94.0,0.0,84.0,118.0,111.0,NaN,NaN,NaN,M0,...,0.000000,0.000000,0.000000,50.0,1404.0,790.0,0.000000,0.000000,0.000000,-8.0,843852.0,-7.0,-83.0,96.725812,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,0,2,yahoo,com,NaN,NaN,NaN,1,12
4,0,50.00,H,4497,514.0,150.0,mastercard,102.0,credit,420.0,87.0,13.0,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,128.0,11.0,463.0,214.0,185.0,245.0,268.0,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,70787.0,29.0,-89.0,100.000000,NotFound,NaN,New,NotFound,166.0,542.0,144.0,New,NotFound,Samsung,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M,5,0,2,google,com,NaN,NaN,Samsung,1,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,0,49.00,W,6550,NaN,150.0,visa,226.0,debit,272.0,87.0,48.0,NaN,NaN,2.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,3.0,2.0,29.0,29.0,30.0,264.0,95.0,56.0,56.0,56.0,T,T,T,M0,...,47.950001,47.950001,47.950001,0.0,0.0,0.0,0.000000,0.000000,0.000000,-8.0,552827.0,26.0,-96.0,100.004713,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,23,1,NaN,NaN,NaN,NaN,NaN,1,6
590536,0,39.50,W,10444,225.0,150.0,mastercard,224.0,debit,204.0,87.0,16.0,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,68.0,13.0,0.0,163.0,0.0,0.0,0.0,T,F,F,M0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,-56.0,445868.0,26.0,-89.0,100.021605,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,23,1,google,com,NaN,NaN,NaN,1,6
590537,0,30.95,W,12037,595.0,150.0,mastercard,224.0,debit,231.0,87.0,36.0,gmail.com,NaN,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,139.0,56.0,0.0,2.0,0.0,0.0,0.0,T,F,F,NaN,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,-96.0,11137.0,26.0,-96.0,99.930493,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [0]:
train3.to_csv('gdrive/My Drive/Colab Notebooks/Fraud/Data/fraud_data_remove_nulls_fill_random_distribution.csv',index=False,compression='gzip')